In [1]:
from __future__ import print_function
from PIL import Image
import os
import os.path
import numpy as np
import sys
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch.utils.data as data

import copy
import hashlib
import errno
import torchvision.transforms as transforms

def check_integrity(fpath, md5):
    if not os.path.isfile(fpath):
        return False
    md5o = hashlib.md5()
    with open(fpath, 'rb') as f:
        # read in 1MB chunks
        for chunk in iter(lambda: f.read(1024 * 1024), b''):
            md5o.update(chunk)
    md5c = md5o.hexdigest()
    if md5c != md5:
        return False
    return True

def download_url(url, root, filename, md5):
    from six.moves import urllib

    root = os.path.expanduser(root)
    fpath = os.path.join(root, filename)

    try:
        os.makedirs(root)
    except OSError as e:
        if e.errno == errno.EEXIST:
            pass
        else:
            raise

    # downloads file
    if os.path.isfile(fpath) and check_integrity(fpath, md5):
        print('Using downloaded and verified file: ' + fpath)
    else:
        try:
            print('Downloading ' + url + ' to ' + fpath)
            urllib.request.urlretrieve(url, fpath)
        except:
            if url[:5] == 'https':
                url = url.replace('https:', 'http:')
                print('Failed download. Trying https -> http instead.'
                      ' Downloading ' + url + ' to ' + fpath)
                urllib.request.urlretrieve(url, fpath)

class CIFAR100(data.Dataset):
    """`CIFAR100 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.

    Args:
        root (string): Root directory of dataset where directory
            ``cifar-10-batches-py`` exists or will be saved to if download is set to True.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.

    """
    base_folder = 'cifar-100-python'
    url = "https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz"
    filename = "cifar-100-python.tar.gz"
    tgz_md5 = 'eb9058c3a382ffc7106e4002c42a8d85'
    train_list = [
        ['train', '16019d7e3df5f24257cddd939b257f8d'],
    ]

    test_list = [
        ['test', 'f0ef6b0ae62326f3e7ffdfab6717acfc'],
    ]
 

    def __init__(self, root, train=True,
                 transform=None,
                 download=False):
        noise_type=None
        target_transform=None
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        self.dataset='cifar100'
        self.noise_type=noise_type
        self.nb_classes=100

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        # now load the picked numpy arrays
        if self.train:
            self.train_data = []
            self.train_labels = []
            for fentry in self.train_list:
                f = fentry[0]
                file = os.path.join(self.root, self.base_folder, f)
                fo = open(file, 'rb')
                if sys.version_info[0] == 2:
                    entry = pickle.load(fo)
                else:
                    entry = pickle.load(fo, encoding='latin1')
                self.train_data.append(entry['data'])
                if 'labels' in entry:
                    self.train_labels += entry['labels']
                else:
                    self.train_labels += entry['fine_labels']
                fo.close()

            self.train_data = np.concatenate(self.train_data)
            self.train_data = self.train_data.reshape((50000, 3, 32, 32))
            self.train_data = self.train_data.transpose((0, 2, 3, 1))  # convert to HWC
            if noise_type is not None:
                # noisify train data
                self.train_labels=np.asarray([[self.train_labels[i]] for i in range(len(self.train_labels))])
                self.train_noisy_labels, self.actual_noise_rate = noisify(dataset=self.dataset, train_labels=self.train_labels, noise_type=noise_type, noise_rate=noise_rate, random_state=random_state, nb_classes=self.nb_classes)
                self.train_noisy_labels=[i[0] for i in self.train_noisy_labels]
                _train_labels=[i[0] for i in self.train_labels]
                self.noise_or_not = np.transpose(self.train_noisy_labels)==np.transpose(_train_labels)
        else:
            f = self.test_list[0][0]
            file = os.path.join(self.root, self.base_folder, f)
            fo = open(file, 'rb')
            if sys.version_info[0] == 2:
                entry = pickle.load(fo)
            else:
                entry = pickle.load(fo, encoding='latin1')
            self.test_data = entry['data']
            if 'labels' in entry:
                self.test_labels = entry['labels']
            else:
                self.test_labels = entry['fine_labels']
            fo.close()
            self.test_data = self.test_data.reshape((10000, 3, 32, 32))
            self.test_data = self.test_data.transpose((0, 2, 3, 1))  # convert to HWC

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            if self.noise_type is not None:
                img, target = self.train_data[index], self.train_noisy_labels[index]
            else:
                img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, index

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_integrity(self):
        root = self.root
        for fentry in (self.train_list + self.test_list):
            filename, md5 = fentry[0], fentry[1]
            fpath = os.path.join(root, self.base_folder, filename)
            if not check_integrity(fpath, md5):
                return False
        return True

    def download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        root = self.root
        download_url(self.url, root, self.filename, self.tgz_md5)

        # extract file
        cwd = os.getcwd()
        tar = tarfile.open(os.path.join(root, self.filename), "r:gz")
        os.chdir(root)
        tar.extractall()
        tar.close()
        os.chdir(cwd)

    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        tmp = 'train' if self.train is True else 'test'
        fmt_str += '    Split: {}\n'.format(tmp)
        fmt_str += '    Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    
])

train=CIFAR100(root='data',download=True,transform=transform_test)
test=CIFAR100(root='data',download=True,train=False,transform=transform_test)

Files already downloaded and verified


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
num_epochs = 20
batch_size = 128
learning_rate = 0.001
print(train)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size,
                                         shuffle=False)


Dataset CIFAR100
    Number of datapoints: 50000
    Split: train
    Root Location: data
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None


In [11]:
model_conv = torchvision.models.resnet101(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 100)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer = torch.optim.Adam(model_conv.fc.parameters(), lr=0.001)


In [12]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images[0] = images[0].to(device)
        images[1] = images[1].to(device)

        # Forward pass
        outputs = model_conv.forward(images[0])
        loss = criterion(outputs, images[1])

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(loss)
       # if (i+1) % 2000 == 0:
        #    print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(100)]
    n_class_samples = [0 for i in range(100)]
    for images in test_loader:
        images[0] = images[0].to(device)
        images[1] = images[1].to(device)
        outputs = model_conv.forward(images[0])
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += images[1].size(0)
        n_correct += (predicted == images[1]).sum().item()
        
        #for i in range(batch_size):
         #   label = images[1][i]
          #  pred = predicted[i]
           # if (label == pred):
            #    n_class_correct[label] += 1
            #n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

tensor(3.0367, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.3744, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.8497, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.9884, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.9254, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.8897, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.7363, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2927, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.4792, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.6083, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3136, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.9627, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.4126, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2673, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.7758, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2619, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.5108, device='cuda:0', grad_fn=<NllLossBackward

In [42]:
predicted=[]
test_loader = torch.utils.data.DataLoader(test, batch_size=128,
                                         shuffle=False)
#model_conv.eval()
for images in test_loader:
        images[0] = images[0].to(device)
        #images[1] = images[1]
        outputs = model_conv.forward(images[0])
        # max returns (value ,index)
        #predicted.append(torch.max(outputs, 1))
        predicted.append(torch.argmax(outputs, 1))
print(len(predicted))
finpred=[]
for i in range(len(predicted)):
  for p in range(len(predicted[i])):
    finpred.append(predicted[i][p])
print(len(finpred)) 
#finpred=[int(i) for i in finpred]
print(finpred)


79
10000
[tensor(12, device='cuda:0'), tensor(33, device='cuda:0'), tensor(90, device='cuda:0'), tensor(91, device='cuda:0'), tensor(83, device='cuda:0'), tensor(56, device='cuda:0'), tensor(33, device='cuda:0'), tensor(79, device='cuda:0'), tensor(73, device='cuda:0'), tensor(57, device='cuda:0'), tensor(16, device='cuda:0'), tensor(99, device='cuda:0'), tensor(90, device='cuda:0'), tensor(9, device='cuda:0'), tensor(51, device='cuda:0'), tensor(21, device='cuda:0'), tensor(92, device='cuda:0'), tensor(66, device='cuda:0'), tensor(70, device='cuda:0'), tensor(53, device='cuda:0'), tensor(70, device='cuda:0'), tensor(73, device='cuda:0'), tensor(48, device='cuda:0'), tensor(96, device='cuda:0'), tensor(21, device='cuda:0'), tensor(16, device='cuda:0'), tensor(39, device='cuda:0'), tensor(22, device='cuda:0'), tensor(45, device='cuda:0'), tensor(32, device='cuda:0'), tensor(5, device='cuda:0'), tensor(61, device='cuda:0'), tensor(41, device='cuda:0'), tensor(73, device='cuda:0'), tensor

In [45]:
import pandas as pd
res = []
for i in range(1,10001):
    res.append([i-1,int(finpred[i-1])])
res = pd.DataFrame(res, columns = ['Id', 'Category']) 
print(res)
res.to_csv('fin.csv', index = False)

        Id  Category
0        0        12
1        1        33
2        2        90
3        3        91
4        4        83
...    ...       ...
9995  9995        83
9996  9996        19
9997  9997        33
9998  9998        42
9999  9999        45

[10000 rows x 2 columns]
